In [1]:
import pandas as pd, numpy as np
import vivarium_output_loader as vol
import lsff_output_processing as lop
import lsff_summarizer
from lsff_summarizer import LSFFOutputSummarizer

!whoami
!date

ndbs
Thu Apr 23 14:55:52 PDT 2020


In [2]:
%load_ext autoreload
%autoreload 2

# Check test runs that added state person time for VAD and NTDs

### Main findings

1. It looks like we have all the expected state person time columns, including for VAD and NTDs. Woo hoo!

2. The `output.hdf` files include the year 2025, but the count space data only has 2020 - 2024.

3. It looks like the vitamin A scaleup decreased vitamin A deficiency, but the folic acid scaleup had no effect on neural tube defects.

4. There is a consistent problem (at least in the count space data), where the prevalences of the different states of a disease add up to more than 1. More details:

    a. For example, in draw 357 in the baseline scenario, the prevalence of `susceptible_to_vitamin_a_deficiency`  in 2020 is 0.736274, and the prevalence of `vitamin_a_deficiency` in 2020 is 0.29047. These two prevalences should add up to 1, but 0.736274+0.29047 = 1.026744.
    
    b. The problem appears in both test runs and for several draws.
    
    c. It definitely happens for vitamin A deficiency, diarrheal diseases, and neural tube defects. I didn't check other causes.
   
    d. There was at least one draw where the single state `susceptible_to_neural_tube_defects` had prevalence greater than 1. Namely, in draw 357 in the baseline scenario, the prevalence of this state in 2020 is 1.026232. This should be impossible.
   
    e. The next thing to do would be to check the `output.hdf` files to see if the corresponding state person time columns add up to the total person time columns. I have not done this yet.
    
    f. I checked the sum of prevalences for two different diseases and got 1.026744 and 1.026748. If we are always getting (nearly) the same number, that could be another clue, but I haven't checked whether the sum is always close to this value.
   
    g. This seems like it is probably a bug, though it's possible there could be something unintuitive happening either with the aggregation over strata or the way we compute prevalence from person time. I have not yet thoroughly considered this possibility.


## 1. Define directories

In [22]:
base_directory = '/share/costeffectiveness/results/vivarium_conic_lsff/00_bugfix/'

vad_path = base_directory + 'vad_prevalence_in_outputs/india_4_22'
ntd_path = base_directory + 'ntd_state_persontime_in_results/india_4_23'

# models_modelruns = {
#     'vad_prevalence_in_outputs': 'india_4_22',
#     'ntd_state_persontime_in_results': 'india_4_23'
# }

In [17]:
# vad_directory = ('/share/costeffectiveness/results/vivarium_conic_lsff/00_bugfix/'
#                 + 'vad_prevalence_in_outputs/india_4_22'
#                  )
# ntd_directory = ('/share/costeffectiveness/results/vivarium_conic_lsff/00_bugfix/'
#                  'ntd_state_persontime_in_results/india_4_23'
#                 )
#     

# Who knew? You can concatenate strings without using +
x = 'a' 'b'
x

'ab'

In [18]:
# But this doesn't work, i.e. you can only use
# string literals, not variables:
# y = x 'c'

## 2. Load `output.hdf` files

In [23]:
vad_raw_output = pd.read_hdf(f'{vad_path}/output.hdf')
vad_raw_output.head()
# raw_output = vol.load_by_location_and_rundate(base_directory, models_modelruns)
# raw_output.head()

total_population_untracked  \
input_draw_number random_seed                               
946.0             35.0                            10510.0   
357.0             35.0                            10446.0   
946.0             21.0                            10478.0   
357.0             45.0                            10526.0   
                  69.0                            10492.0   

                               total_population_tracked  total_population  \
input_draw_number random_seed                                               
946.0             35.0                          10048.0           20558.0   
357.0             35.0                           9847.0           20293.0   
946.0             21.0                           9771.0           20249.0   
357.0             45.0                           9882.0           20408.0   
                  69.0                           9907.0           20399.0   

                               diarrheal_diseases_prevalent_cases_at_sim_end  \
input_draw_number random_seed                                                  
946.0             35.0                                                2330.0   
357.0             35.0                                                2192.0   
946.0             21.0                                                2190.0   
357.0             45.0                                                2209.0   
                  69.0                                                2078.0   

                               susceptible_to_diarrheal_diseases_event_count  \
input_draw_number random_seed                                                  
946.0             35.0                                                9990.0   
357.0             35.0                                                9713.0   
946.0             21.0                                                9727.0   
357.0             45.0                                                9830.0   
                  69.0                                                9248.0   

                               diarrheal_diseases_event_count  \
input_draw_number random_seed                                   
946.0             35.0                                12967.0   
357.0             35.0                                12624.0   
946.0             21.0                                12555.0   
357.0             45.0                                12723.0   
                  69.0                                11947.0   

                               measles_prevalent_cases_at_sim_end  \
input_draw_number random_seed                                       
946.0             35.0                                      786.0   
357.0             35.0                                      345.0   
946.0             21.0                                      733.0   
357.0             45.0                                      312.0   
                  69.0                                      272.0   

                               susceptible_to_measles_event_count  \
input_draw_number random_seed                                       
946.0             35.0                                        0.0   
357.0             35.0                                        0.0   
946.0             21.0                                        0.0   
357.0             45.0                                        0.0   
                  69.0                                        0.0   

                               measles_event_count  \
input_draw_number random_seed                        
946.0             35.0                       882.0   
357.0             35.0                       396.0   
946.0             21.0                       849.0   
357.0             45.0                       357.0   
                  69.0                       312.0   

                               recovered_from_measles_event_count  ...  \
input_draw_number random_seed                                   

In [24]:
ntd_raw_output = pd.read_hdf(f'{ntd_path}/output.hdf')
ntd_raw_output.head()

total_population_untracked  \
input_draw_number random_seed                               
357.0             44.0                            10430.0   
946.0             35.0                            10510.0   
357.0             69.0                            10471.0   
602.0             21.0                            10440.0   
                  69.0                            10511.0   

                               total_population_tracked  total_population  \
input_draw_number random_seed                                               
357.0             44.0                           9923.0           20353.0   
946.0             35.0                          10048.0           20558.0   
357.0             69.0                           9928.0           20399.0   
602.0             21.0                           9901.0           20341.0   
                  69.0                          10000.0           20511.0   

                               diarrheal_diseases_prevalent_cases_at_sim_end  \
input_draw_number random_seed                                                  
357.0             44.0                                                2214.0   
946.0             35.0                                                2330.0   
357.0             69.0                                                2260.0   
602.0             21.0                                                2094.0   
                  69.0                                                2239.0   

                               susceptible_to_diarrheal_diseases_event_count  \
input_draw_number random_seed                                                  
357.0             44.0                                                9617.0   
946.0             35.0                                                9990.0   
357.0             69.0                                                9872.0   
602.0             21.0                                                9463.0   
                  69.0                                                9642.0   

                               diarrheal_diseases_event_count  \
input_draw_number random_seed                                   
357.0             44.0                                12481.0   
946.0             35.0                                12967.0   
357.0             69.0                                12780.0   
602.0             21.0                                12305.0   
                  69.0                                12558.0   

                               measles_prevalent_cases_at_sim_end  \
input_draw_number random_seed                                       
357.0             44.0                                      332.0   
946.0             35.0                                      786.0   
357.0             69.0                                      329.0   
602.0             21.0                                     1520.0   
                  69.0                                     1567.0   

                               susceptible_to_measles_event_count  \
input_draw_number random_seed                                       
357.0             44.0                                        0.0   
946.0             35.0                                        0.0   
357.0             69.0                                        0.0   
602.0             21.0                                        0.0   
                  69.0                                        0.0   

                               measles_event_count  \
input_draw_number random_seed                        
357.0             44.0                       378.0   
946.0             35.0                       882.0   
357.0             69.0                       375.0   
602.0             21.0                      1713.0   
                  69.0                      1755.0   

                               recovered_from_measles_event_count  ...  \
input_draw_number random_seed                                   

## 3. Load count space data

In [26]:
vad_data = vol.load_transformed_count_data(f'{vad_path}/count_data')
vad_data.keys()

dict_keys(['gestational_age', 'transition_count', 'deaths', 'state_person_time', 'births_with_ntd', 'population', 'person_time', 'ylls', 'ylds', 'births', 'birth_weight'])

In [27]:
ntd_data = vol.load_transformed_count_data(f'{ntd_path}/count_data')
ntd_data.keys()

dict_keys(['gestational_age', 'transition_count', 'deaths', 'state_person_time', 'births_with_ntd', 'population', 'person_time', 'ylls', 'ylds', 'births', 'birth_weight'])

## 4. Print column reports for raw output

In [29]:
vad_output = LSFFOutputSummarizer(vad_raw_output)
vad_output.print_column_report()

Number of data columns in output: 14501
Total number of columns captured in categories: 14501

Number of columns in each category:
 {'input_draw': 1, 'random_seed': 1, 'location': 0, 'intervention': 1, 'run_time': 0, 'diseases_at_end': 4, 'transition_count': 3465, 'population': 5, 'person_time': 432, 'state_person_time': 3456, 'treated_days': 0, 'mortality': 2160, 'total_daly': 2, 'yld': 2592, 'yll': 2160, 'categorical_risk': 0, 'birth_prevalence': 108, 'live_births': 108, 'proportion': 2, 'distribution_mean': 2, 'distribution_std_dev': 2, 'prevalence': 0} 

Missing (0 data column(s) not captured in a category):
 set()

Repeated (0 data column(s) appearing in more than one category):
 {}

Empty categories (5 categories with no matching data columns):
 ['location', 'run_time', 'treated_days', 'categorical_risk', 'prevalence']


In [30]:
ntd_output = LSFFOutputSummarizer(ntd_raw_output)
ntd_output.print_column_report()

Number of data columns in output: 15365
Total number of columns captured in categories: 15365

Number of columns in each category:
 {'input_draw': 1, 'random_seed': 1, 'location': 0, 'intervention': 1, 'run_time': 0, 'diseases_at_end': 4, 'transition_count': 3465, 'population': 5, 'person_time': 432, 'state_person_time': 4320, 'treated_days': 0, 'mortality': 2160, 'total_daly': 2, 'yld': 2592, 'yll': 2160, 'categorical_risk': 0, 'birth_prevalence': 108, 'live_births': 108, 'proportion': 2, 'distribution_mean': 2, 'distribution_std_dev': 2, 'prevalence': 0} 

Missing (0 data column(s) not captured in a category):
 set()

Repeated (0 data column(s) appearing in more than one category):
 {}

Empty categories (5 categories with no matching data columns):
 ['location', 'run_time', 'treated_days', 'categorical_risk', 'prevalence']


## 5. See what the scenario columns look like

In [36]:
vad_output.subdata('intervention').head(10)

fortification_intervention.scenario
input_draw_number random_seed                                    
946.0             35.0                                   baseline
357.0             35.0                                   baseline
946.0             21.0                                   baseline
357.0             45.0          folic_acid_fortification_scale_up
                  69.0           vitamin_a_fortification_scale_up
946.0             69.0           vitamin_a_fortification_scale_up
357.0             12.0                                   baseline
602.0             35.0          folic_acid_fortification_scale_up
946.0             12.0           vitamin_a_fortification_scale_up
                  22.0          folic_acid_fortification_scale_up

In [37]:
ntd_output.subdata('intervention').head(10)

fortification_intervention.scenario
input_draw_number random_seed                                    
357.0             44.0                                   baseline
946.0             35.0          folic_acid_fortification_scale_up
357.0             69.0          folic_acid_fortification_scale_up
602.0             21.0                                   baseline
                  69.0                                   baseline
946.0             22.0          folic_acid_fortification_scale_up
                  22.0           vitamin_a_fortification_scale_up
602.0             22.0          folic_acid_fortification_scale_up
357.0             45.0                                   baseline
946.0             45.0                                   baseline

## 6. Check state person time columns for both runs

It looks like this is the only category where there's a difference

In [31]:
# Total number of non-state-person-time columns for the two models is equal
(15365 - 4320) == (14501 - 3456)

True

In [45]:
# There are 864 more columns in the NTD output (all in state person time)
4320 - 3456

864

In [63]:
# 864 = 2^5 * 3^3
864/4/4/2/3/3/3

1.0

In [67]:
2**5 * 3**3

864

In [80]:
# 2 NTD states x 5 years x 2 sexes x 4 age groups x 3 FA coverage groups x 3 VA coverage groups
2*5*2*4*3*3

720

So there are 144 extra columns...

In [76]:
864-720

144

### Try to find the 144 columns unaccounted for in `ntd_output`

Ahh, there are 6 years, not 5. But the count space data only has 5 years (2020 - 2024) -- see below.

In [78]:
ntd_output.subdata('state_person_time').filter(regex='neural_tube_defects').head()

susceptible_to_neural_tube_defects_person_time_in_2020_among_male_in_age_group_early_neonatal_folic_acid_unknown_vitamin_a_uncovered  \
input_draw_number random_seed                                                                                                                                         
357.0             44.0                                                  4.982888                                                                                      
946.0             35.0                                                  2.989733                                                                                      
357.0             69.0                                                  4.982888                                                                                      
602.0             21.0                                                  5.481177                                                                                      
                  69.0                                                  6.976044                                                                                      

                               susceptible_to_neural_tube_defects_person_time_in_2020_among_female_in_age_group_early_neonatal_folic_acid_unknown_vitamin_a_uncovered  \
input_draw_number random_seed                                                                                                                                           
357.0             44.0                                                  5.481177                                                                                        
946.0             35.0                                                  3.986311                                                                                        
357.0             69.0                                                  4.484600                                                                                        
602.0             21.0                                                  4.484600                                                                                        
                  69.0                                                  3.986311                                                                                        

                               susceptible_to_neural_tube_defects_person_time_in_2020_among_male_in_age_group_late_neonatal_folic_acid_unknown_vitamin_a_uncovered  \
input_draw_number random_seed                                                                                                                                        
357.0             44.0                                                 14.450376                                                                                     
946.0             35.0                                                 24.416153                                                                                     
357.0             69.0                                                 23.419576                                                                                     
602.0             21.0                                                 19.433265                                                                                     
                  69.0                                                 24.416153                                                                                     

                               susceptible_to_neural_tube_defects_person_time_in_2020_among_female_in_age_group_late_neonatal_folic_acid_unknown_vitamin_a_uncovered  \
input_draw_number random_seed                                                                                                                                          
357.0             44.0                                                 17.938398                                                                                       
946.0     

Aha! The raw output includes 2025, so there are 6 years, not 5 like in the count space data.

In [81]:
# 2 NTD states x 6 years x 2 sexes x 4 age groups x 3 FA coverage groups x 3 VA coverage groups
2*6*2*4*3*3

864

In [33]:
vad_output.columns('state_person_time')[-10:]

['susceptible_to_vitamin_a_deficiency_person_time_in_2025_among_male_in_age_group_1_to_4_folic_acid_covered_vitamin_a_effectively_covered',
 'susceptible_to_vitamin_a_deficiency_person_time_in_2025_among_female_in_age_group_1_to_4_folic_acid_covered_vitamin_a_effectively_covered',
 'vitamin_a_deficiency_person_time_in_2025_among_male_in_age_group_early_neonatal_folic_acid_covered_vitamin_a_effectively_covered',
 'vitamin_a_deficiency_person_time_in_2025_among_female_in_age_group_early_neonatal_folic_acid_covered_vitamin_a_effectively_covered',
 'vitamin_a_deficiency_person_time_in_2025_among_male_in_age_group_late_neonatal_folic_acid_covered_vitamin_a_effectively_covered',
 'vitamin_a_deficiency_person_time_in_2025_among_female_in_age_group_late_neonatal_folic_acid_covered_vitamin_a_effectively_covered',
 'vitamin_a_deficiency_person_time_in_2025_among_male_in_age_group_post_neonatal_folic_acid_covered_vitamin_a_effectively_covered',
 'vitamin_a_deficiency_person_time_in_2025_among_fem

In [34]:
ntd_output.columns('state_person_time')[-10:]

['susceptible_to_neural_tube_defects_person_time_in_2025_among_male_in_age_group_1_to_4_folic_acid_covered_vitamin_a_effectively_covered',
 'susceptible_to_neural_tube_defects_person_time_in_2025_among_female_in_age_group_1_to_4_folic_acid_covered_vitamin_a_effectively_covered',
 'neural_tube_defects_person_time_in_2025_among_male_in_age_group_early_neonatal_folic_acid_covered_vitamin_a_effectively_covered',
 'neural_tube_defects_person_time_in_2025_among_female_in_age_group_early_neonatal_folic_acid_covered_vitamin_a_effectively_covered',
 'neural_tube_defects_person_time_in_2025_among_male_in_age_group_late_neonatal_folic_acid_covered_vitamin_a_effectively_covered',
 'neural_tube_defects_person_time_in_2025_among_female_in_age_group_late_neonatal_folic_acid_covered_vitamin_a_effectively_covered',
 'neural_tube_defects_person_time_in_2025_among_male_in_age_group_post_neonatal_folic_acid_covered_vitamin_a_effectively_covered',
 'neural_tube_defects_person_time_in_2025_among_female_in_a

In [42]:
vad_output.subdata('state_person_time').head()

susceptible_to_diarrheal_diseases_person_time_in_2020_among_male_in_age_group_early_neonatal_folic_acid_unknown_vitamin_a_uncovered  \
input_draw_number random_seed                                                                                                                                        
946.0             35.0                                                  2.989733                                                                                     
357.0             35.0                                                  7.474333                                                                                     
946.0             21.0                                                  9.467488                                                                                     
357.0             45.0                                                  7.474333                                                                                     
                  69.0                                                  0.996578                                                                                     

                               susceptible_to_diarrheal_diseases_person_time_in_2020_among_female_in_age_group_early_neonatal_folic_acid_unknown_vitamin_a_uncovered  \
input_draw_number random_seed                                                                                                                                          
946.0             35.0                                                  3.488022                                                                                       
357.0             35.0                                                 11.958932                                                                                       
946.0             21.0                                                  5.481177                                                                                       
357.0             45.0                                                  9.965777                                                                                       
                  69.0                                                  0.498289                                                                                       

                               susceptible_to_diarrheal_diseases_person_time_in_2020_among_male_in_age_group_late_neonatal_folic_acid_unknown_vitamin_a_uncovered  \
input_draw_number random_seed                                                                                                                                       
946.0             35.0                                                 24.416153                                                                                    
357.0             35.0                                                 20.429843                                                                                    
946.0             21.0                                                 23.917864                                                                                    
357.0             45.0                                                 22.422998                                                                                    
                  69.0                                                  3.986311                                                                                    

                               susceptible_to_diarrheal_diseases_person_time_in_2020_among_female_in_age_group_late_neonatal_folic_acid_unknown_vitamin_a_uncovered  \
input_draw_number random_seed                                                                                                                                         
946.0             35.0                                                 19.931554                                                                                      
357.0             35.0            

## 7. Load state person time from count data

Ok, so it looks like the difference is that state person time for neural tube defects has been added in the second model, which is what we expect.

In [40]:
vad_spt = vad_data['state_person_time']
vad_spt.head()

,year,age_group,sex,cause,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,357,baseline,0.0
1,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,357,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,357,vitamin_a_fortification_scale_up,0.0
3,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,602,baseline,0.0
4,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,602,folic_acid_fortification_scale_up,0.0


In [64]:
vad_spt.shape

(25920, 10)

In [83]:
# Note there are only 5 years in count space data, but 6 in the raw output
print([vad_spt[col].nunique() for col in vad_spt.columns])

[5, 4, 2, 8, 3, 3, 1, 3, 3, 4025]


In [86]:
# Check that product of identification columns equals total length of dataframe
prod = 1
for col in vad_spt.columns[:-1]:
    prod *= vad_spt[col].nunique()
prod

25920

In [73]:
# 8 cause states, 3 input draws
5*4*2*8*3*3*1*3*3

25920

In [41]:
ntd_spt = ntd_data['state_person_time']
ntd_spt.head()

,year,age_group,sex,cause,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,357,baseline,0.0
1,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,357,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,357,vitamin_a_fortification_scale_up,0.0
3,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,602,baseline,0.0
4,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,602,folic_acid_fortification_scale_up,0.0


In [65]:
ntd_spt.shape

(32400, 10)

In [72]:
# 10 cause states, 3 input draws
5*4*2*10*3*3*1*3*3

32400

In [75]:
5*4*2*10*3*3 - 5*4*2*8*3*3

720

In [43]:
vad_spt.cause.unique()

array(['diarrheal_diseases', 'lower_respiratory_infections', 'measles',
       'susceptible_to_diarrheal_diseases',
       'susceptible_to_lower_respiratory_infections',
       'susceptible_to_measles', 'susceptible_to_vitamin_a_deficiency',
       'vitamin_a_deficiency'], dtype=object)

In [44]:
ntd_spt.cause.unique()

array(['diarrheal_diseases', 'lower_respiratory_infections', 'measles',
       'neural_tube_defects', 'susceptible_to_diarrheal_diseases',
       'susceptible_to_lower_respiratory_infections',
       'susceptible_to_measles', 'susceptible_to_neural_tube_defects',
       'susceptible_to_vitamin_a_deficiency', 'vitamin_a_deficiency'],
      dtype=object)

In [50]:
ntd_spt.vitamin_a_fortification_group.unique()

array(['covered', 'effectively_covered', 'uncovered'], dtype=object)

In [51]:
ntd_spt.folic_acid_fortification_group.unique()

array(['covered', 'uncovered', 'unknown'], dtype=object)

In [52]:
ntd_spt.age_group.unique()

array(['1_to_4', 'early_neonatal', 'late_neonatal', 'post_neonatal'],
      dtype=object)

In [54]:
ntd_spt.year.unique()

array(['2020', '2021', '2022', '2023', '2024'], dtype=object)

In [57]:
ntd_spt.query('folic_acid_fortification_group == "unknown"').cause.unique()

array(['diarrheal_diseases', 'lower_respiratory_infections', 'measles',
       'neural_tube_defects', 'susceptible_to_diarrheal_diseases',
       'susceptible_to_lower_respiratory_infections',
       'susceptible_to_measles', 'susceptible_to_neural_tube_defects',
       'susceptible_to_vitamin_a_deficiency', 'vitamin_a_deficiency'],
      dtype=object)

## 8. Compute VAD prevalence

There seems to be a problem where the prevalences of all states for a given disease add up to more than 1.

In [89]:
vad_pt = vad_data['person_time']
vad_pt.head()

,year,age_group,sex,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,female,covered,covered,person_time,357,baseline,0.0
1,2020,1_to_4,female,covered,covered,person_time,357,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,female,covered,covered,person_time,357,vitamin_a_fortification_scale_up,0.0
3,2020,1_to_4,female,covered,covered,person_time,602,baseline,0.0
4,2020,1_to_4,female,covered,covered,person_time,602,folic_acid_fortification_scale_up,0.0


### Divide state person time by person time to get prevalence

Stratify only by year in both numerator and denominator (i.e. aggregate over all other stratification variables), and broadcast over cause states to compute all prevalences at once. Use a multiplier of 1 to just get ratios in [0,1].

In [95]:
prevalence = lop.rate_or_ratio(vad_spt, vad_pt, ['year'], ['year'], 1, ['cause'])
prevalence.head()

,year,input_draw,scenario,cause,value
0,2020,357,baseline,diarrheal_diseases,0.174887
1,2020,357,baseline,lower_respiratory_infections,0.029063
2,2020,357,baseline,measles,0.004404
3,2020,357,baseline,susceptible_to_diarrheal_diseases,0.851862
4,2020,357,baseline,susceptible_to_lower_respiratory_infections,0.997686


In [107]:
prevalence.year.unique()

array(['2020', '2021', '2022', '2023', '2024'], dtype=object)

### The prevalence of VAD in the "susceptible" and "with condition" groups adds to more than 1

I think that shouldn't happen, unless there's something unintuitive happening either (1) with the aggregation over strata or (2) with the fact that we're using ratios of person time to get average point prevalences. I don't think either of those things should change the fact that the prevalences should add up to 1, but I'd have to think about it more carefully to be sure. 

In [103]:
prevalence[prevalence.cause.str.contains('vitamin_a_deficiency')].head()

,year,input_draw,scenario,cause,value
6,2020,357,baseline,susceptible_to_vitamin_a_deficiency,0.736274
7,2020,357,baseline,vitamin_a_deficiency,0.290475
14,2020,357,folic_acid_fortification_scale_up,susceptible_to_vitamin_a_deficiency,0.736274
15,2020,357,folic_acid_fortification_scale_up,vitamin_a_deficiency,0.290475
22,2020,357,vitamin_a_fortification_scale_up,susceptible_to_vitamin_a_deficiency,0.863951


In [132]:
vad_prevalence = prevalence[prevalence.cause.str.contains('vitamin_a_deficiency')]
vad_prevalence.head()

,year,input_draw,scenario,cause,value
8,2020,357,baseline,susceptible_to_vitamin_a_deficiency,0.736274
9,2020,357,baseline,vitamin_a_deficiency,0.290475
18,2020,357,folic_acid_fortification_scale_up,susceptible_to_vitamin_a_deficiency,0.736274
19,2020,357,folic_acid_fortification_scale_up,vitamin_a_deficiency,0.290475
28,2020,357,vitamin_a_fortification_scale_up,susceptible_to_vitamin_a_deficiency,0.863951


### Since there are only 3 draws, we can unstack them to see all the values

In [135]:
vad_prevalence.set_index(['year', 'input_draw', 'scenario', 'cause']).unstack('input_draw')

value  \
input_draw                                                                       357   
year scenario                          cause                                           
2020 baseline                          susceptible_to_vitamin_a_deficiency  0.736274   
                                       vitamin_a_deficiency                 0.290475   
     folic_acid_fortification_scale_up susceptible_to_vitamin_a_deficiency  0.736274   
                                       vitamin_a_deficiency                 0.290475   
     vitamin_a_fortification_scale_up  susceptible_to_vitamin_a_deficiency  0.863951   
                                       vitamin_a_deficiency                 0.162490   
2021 baseline                          susceptible_to_vitamin_a_deficiency  0.700229   
                                       vitamin_a_deficiency                 0.291927   
     folic_acid_fortification_scale_up susceptible_to_vitamin_a_deficiency  0.700229   
                                       vitamin_a_deficiency                 0.291927   
     vitamin_a_fortification_scale_up  susceptible_to_vitamin_a_deficiency  0.816746   
                                       vitamin_a_deficiency                 0.175535   
2022 baseline                          susceptible_to_vitamin_a_deficiency  0.694247   
                                       vitamin_a_deficiency                 0.298924   
     folic_acid_fortification_scale_up susceptible_to_vitamin_a_deficiency  0.694247   
                                       vitamin_a_deficiency                 0.298924   
     vitamin_a_fortification_scale_up  susceptible_to_vitamin_a_deficiency  0.811082   
                                       vitamin_a_deficiency                 0.182149   
2023 baseline                          susceptible_to_vitamin_a_deficiency  0.691038   
                                       vitamin_a_deficiency                 0.300220   
     folic_acid_fortification_scale_up susceptible_to_vitamin_a_deficiency  0.691038   
                                       vitamin_a_deficiency                 0.300220   
     vitamin_a_fortification_scale_up  susceptible_to_vitamin_a_deficiency  0.808451   
                                       vitamin_a_deficiency                 0.182777   
2024 baseline                          susceptible_to_vitamin_a_deficiency  0.693494   
                                       vitamin_a_deficiency                 0.299712   
     folic_acid_fortification_scale_up susceptible_to_vitamin_a_deficiency  0.693494   
                                       vitamin_a_deficiency                 0.299712   
     vitamin_a_fortification_scale_up  susceptible_to_vitamin_a_deficiency  0.814042   
                                       vitamin_a_deficiency                 0.179317   

                                                                                      \
input_draw                                                                       602   
year scenario                          cause                                           
2020 baseline                          susceptible_to_vitamin_a_deficiency  0.701943   
                                       vitamin_a_deficiency                 0.325787   
     folic_acid_fortification_scale_up susceptible_to_vitamin_a_deficiency  0.701943   
                                       vitamin_a_deficiency                 0.325787   
     vitamin_a_fortification_scale_up  susceptible_to_vitamin_a_deficiency  0.812798   
                                       vitamin_a_deficiency                 0.214571   
2021 baseline                          susceptible_to_vitamin_a_deficiency  0.662786   
                                       vitamin_a_deficiency                 0.329272   
     folic_acid_fortification_scale_up susceptible_to_vitamin_a_deficiency  0.662786   
                                       vitamin_a_deficiency                 0.329272   
     vitamin_a_fortification_s

In [112]:
vad_prevalence.year.unique()

array(['2020', '2021', '2022', '2023', '2024'], dtype=object)

### Using describe, the min, median, and max will be the values of the 3 draws

It looks like the VA scaleup decreased prevalence of VAD, which is good.

In [133]:
vad_prevalence.groupby(['year', 'scenario', 'cause']).value.describe(percentiles=[])

count  \
year scenario                          cause                                        
2020 baseline                          susceptible_to_vitamin_a_deficiency    3.0   
                                       vitamin_a_deficiency                   3.0   
     folic_acid_fortification_scale_up susceptible_to_vitamin_a_deficiency    3.0   
                                       vitamin_a_deficiency                   3.0   
     vitamin_a_fortification_scale_up  susceptible_to_vitamin_a_deficiency    3.0   
                                       vitamin_a_deficiency                   3.0   
2021 baseline                          susceptible_to_vitamin_a_deficiency    3.0   
                                       vitamin_a_deficiency                   3.0   
     folic_acid_fortification_scale_up susceptible_to_vitamin_a_deficiency    3.0   
                                       vitamin_a_deficiency                   3.0   
     vitamin_a_fortification_scale_up  susceptible_to_vitamin_a_deficiency    3.0   
                                       vitamin_a_deficiency                   3.0   
2022 baseline                          susceptible_to_vitamin_a_deficiency    3.0   
                                       vitamin_a_deficiency                   3.0   
     folic_acid_fortification_scale_up susceptible_to_vitamin_a_deficiency    3.0   
                                       vitamin_a_deficiency                   3.0   
     vitamin_a_fortification_scale_up  susceptible_to_vitamin_a_deficiency    3.0   
                                       vitamin_a_deficiency                   3.0   
2023 baseline                          susceptible_to_vitamin_a_deficiency    3.0   
                                       vitamin_a_deficiency                   3.0   
     folic_acid_fortification_scale_up susceptible_to_vitamin_a_deficiency    3.0   
                                       vitamin_a_deficiency                   3.0   
     vitamin_a_fortification_scale_up  susceptible_to_vitamin_a_deficiency    3.0   
                                       vitamin_a_deficiency                   3.0   
2024 baseline                          susceptible_to_vitamin_a_deficiency    3.0   
                                       vitamin_a_deficiency                   3.0   
     folic_acid_fortification_scale_up susceptible_to_vitamin_a_deficiency    3.0   
                                       vitamin_a_deficiency                   3.0   
     vitamin_a_fortification_scale_up  susceptible_to_vitamin_a_deficiency    3.0   
                                       vitamin_a_deficiency                   3.0   

                                                                                mean  \
year scenario                          cause                                           
2020 baseline                          susceptible_to_vitamin_a_deficiency  0.718360   
                                       vitamin_a_deficiency                 0.308481   
     folic_acid_fortification_scale_up susceptible_to_vitamin_a_deficiency  0.718360   
                                       vitamin_a_deficiency                 0.308481   
     vitamin_a_fortification_scale_up  susceptible_to_vitamin_a_deficiency  0.813643   
                                       vitamin_a_deficiency                 0.212942   
2021 baseline                          susceptible_to_vitamin_a_deficiency  0.681818   
                                       vitamin_a_deficiency                 0.310605   
     folic_acid_fortification_scale_up susceptible_to_vitamin_a_deficiency  0.681818   
                                       vitamin_a_deficiency                 0.310605   
     vitamin_a_fortification_scale_up  susceptible_to_vitamin_a_deficiency  0.768795   
                                       vitamin_a_deficiency                 0.223739   
2022 baseline                          susceptible_to_vitamin_a_deficiency  0.678365   
                                   

In [121]:
vad_prevalence.cause.unique()

array(['vitamin_a_deficiency'], dtype=object)

## 9. Compute prevalence of neural tube defects

1. As in the VAD output, there seems to be a problem where the prevalences of all states for a given disease add up to more than 1.
2. Also, there is at least one state whose prevalence all by itselfs is greater than 1.
3. It appears that FA fortification scaleup didn't have an effect - the numbers are the same in the baseline and intervention scenarios.

In [126]:
ntd_spt.head()

,year,age_group,sex,cause,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,357,baseline,0.0
1,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,357,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,357,vitamin_a_fortification_scale_up,0.0
3,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,602,baseline,0.0
4,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,602,folic_acid_fortification_scale_up,0.0


In [125]:
ntd_pt = ntd_data['person_time']
ntd_pt.head()

,year,age_group,sex,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,female,covered,covered,person_time,357,baseline,0.0
1,2020,1_to_4,female,covered,covered,person_time,357,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,female,covered,covered,person_time,357,vitamin_a_fortification_scale_up,0.0
3,2020,1_to_4,female,covered,covered,person_time,602,baseline,0.0
4,2020,1_to_4,female,covered,covered,person_time,602,folic_acid_fortification_scale_up,0.0


### Divide state person time by person time to get prevalence

Stratify only by year in both numerator and denominator (i.e. aggregate over all other stratification variables), and broadcast over cause states to compute all prevalences at once. Use a multiplier of 1 to just get ratios in [0,1].

In [117]:
prevalence = lop.rate_or_ratio(ntd_spt, ntd_pt, ['year'], ['year'], 1, ['cause'])
prevalence.head()

,year,input_draw,scenario,cause,value
0,2020,357,baseline,diarrheal_diseases,0.174887
1,2020,357,baseline,lower_respiratory_infections,0.029063
2,2020,357,baseline,measles,0.004404
3,2020,357,baseline,neural_tube_defects,0.000516
4,2020,357,baseline,susceptible_to_diarrheal_diseases,0.851862


### Prevalences of "with condition" and "susceptible" add up to more than 1 for diarrheal diseases

Same problem we had with VAD above. This looks like a consistent pattern.

In [128]:
prevalence[prevalence.cause.str.contains('diarrheal_diseases')].head()

,year,input_draw,scenario,cause,value
0,2020,357,baseline,diarrheal_diseases,0.174887
4,2020,357,baseline,susceptible_to_diarrheal_diseases,0.851862
10,2020,357,folic_acid_fortification_scale_up,diarrheal_diseases,0.174887
14,2020,357,folic_acid_fortification_scale_up,susceptible_to_diarrheal_diseases,0.851862
20,2020,357,vitamin_a_fortification_scale_up,diarrheal_diseases,0.158537


### Prevalence of "susceptible to neural tube defects" by itself is greater than 1 in draw 357 -- that's no good

In [122]:
ntd_prevalence = prevalence[prevalence.cause.str.contains('neural_tube_defects')]
ntd_prevalence.head()

,year,input_draw,scenario,cause,value
3,2020,357,baseline,neural_tube_defects,0.000516
7,2020,357,baseline,susceptible_to_neural_tube_defects,1.026232
13,2020,357,folic_acid_fortification_scale_up,neural_tube_defects,0.000516
17,2020,357,folic_acid_fortification_scale_up,susceptible_to_neural_tube_defects,1.026232
23,2020,357,vitamin_a_fortification_scale_up,neural_tube_defects,0.000516


In [131]:
ntd_prevalence.groupby(['year', 'scenario', 'cause']).value.describe(percentiles=[])

count  \
year scenario                          cause                                       
2020 baseline                          neural_tube_defects                   3.0   
                                       susceptible_to_neural_tube_defects    3.0   
     folic_acid_fortification_scale_up neural_tube_defects                   3.0   
                                       susceptible_to_neural_tube_defects    3.0   
     vitamin_a_fortification_scale_up  neural_tube_defects                   3.0   
                                       susceptible_to_neural_tube_defects    3.0   
2021 baseline                          neural_tube_defects                   3.0   
                                       susceptible_to_neural_tube_defects    3.0   
     folic_acid_fortification_scale_up neural_tube_defects                   3.0   
                                       susceptible_to_neural_tube_defects    3.0   
     vitamin_a_fortification_scale_up  neural_tube_defects                   3.0   
                                       susceptible_to_neural_tube_defects    3.0   
2022 baseline                          neural_tube_defects                   3.0   
                                       susceptible_to_neural_tube_defects    3.0   
     folic_acid_fortification_scale_up neural_tube_defects                   3.0   
                                       susceptible_to_neural_tube_defects    3.0   
     vitamin_a_fortification_scale_up  neural_tube_defects                   3.0   
                                       susceptible_to_neural_tube_defects    3.0   
2023 baseline                          neural_tube_defects                   3.0   
                                       susceptible_to_neural_tube_defects    3.0   
     folic_acid_fortification_scale_up neural_tube_defects                   3.0   
                                       susceptible_to_neural_tube_defects    3.0   
     vitamin_a_fortification_scale_up  neural_tube_defects                   3.0   
                                       susceptible_to_neural_tube_defects    3.0   
2024 baseline                          neural_tube_defects                   3.0   
                                       susceptible_to_neural_tube_defects    3.0   
     folic_acid_fortification_scale_up neural_tube_defects                   3.0   
                                       susceptible_to_neural_tube_defects    3.0   
     vitamin_a_fortification_scale_up  neural_tube_defects                   3.0   
                                       susceptible_to_neural_tube_defects    3.0   

                                                                               mean  \
year scenario                          cause                                          
2020 baseline                          neural_tube_defects                 0.000519   
                                       susceptible_to_neural_tube_defects  1.026322   
     folic_acid_fortification_scale_up neural_tube_defects                 0.000519   
                                       susceptible_to_neural_tube_defects  1.026322   
     vitamin_a_fortification_scale_up  neural_tube_defects                 0.000519   
                                       susceptible_to_neural_tube_defects  1.026066   
2021 baseline                          neural_tube_defects                 0.000451   
                                       susceptible_to_neural_tube_defects  0.991972   
     folic_acid_fortification_scale_up neural_tube_defects                 0.000451   
                                       susceptible_to_neural_tube_defects  0.991972   
     vitamin_a_fortification_scale_up  neural_tube_defects                 0.000451   
                                       susceptible_to_neural_tube_defects  0.992084   
2022 baseline                          neural_tube_defects                 0.000427   
                                       susceptible_to_neural_tube_defects  0.9925